In [3]:
#!/usr/bin/env python3
"""
2-fold BH-FDR discovery on train, validation on opposite fold (leak-free).

Pipeline (per fold f∈{1,2}):
  - Fit PCA on TRAIN_f only (ViT and brain). n_components by 90% var on TRAIN_f.
  - Compute ALL ViT-PC_i × Brain-PC_j correlations on TRAIN_f; BH-FDR at alpha=0.05.
  - Validate ONLY those pairs on TEST_(!f), projecting TEST into TRAIN_f bases.
Repeat for both folds; then keep pairs that discover+validate in BOTH directions.

Outputs:
  - raw_fold_results.csv : per-pair per-direction stats
  - fold_summary.csv      : summary per fold
  - replicated_pairs.csv  : pairs that replicate both ways (by fold-local indices)
"""

import numpy as np
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from scipy.special import softmax
from scipy.stats import pearsonr
from skbio.stats.composition import clr
from statsmodels.stats.multitest import multipletests

# -----------------------------
# CONFIG
# -----------------------------
VIT_PATH    = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'

AREA_NAME   = 'VISam'   # change as needed
N_IMAGES, N_TRIALS = 118, 50
ALPHA_FDR   = 0.05
N_SPLITS    = 5
RANDOM_STATE= 42

# -----------------------------
# LOAD DATA
# -----------------------------
print("🔹 Loading ViT embeddings...")
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']   # (images × D_vit)
vit_clr = clr(softmax(np.asarray(vit_logits), axis=1) + 1e-12)

print(f"🔹 Loading neural responses ({AREA_NAME})...")
dat   = np.load(NEURAL_PATH, mmap_mode='r')
areas = np.load(AREAS_PATH, allow_pickle=True)
mask  = areas == AREA_NAME
dat   = dat[mask]
n_neurons, n_total = dat.shape
n_time = n_total // (N_IMAGES * N_TRIALS)
dat = dat.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)
X_mean = dat.mean(axis=(2,3))                       # (neurons × images)
print(f"{AREA_NAME}: {n_neurons} neurons × {N_IMAGES} images")

# -----------------------------
# HELPERS
# -----------------------------
def fit_pca_train_only(vit_train, brain_train, seed=RANDOM_STATE):
    # ViT PCA
    pca_vit = PCA(random_state=seed).fit(vit_train)
    vit_cum = np.cumsum(pca_vit.explained_variance_ratio_)
    vit_n   = int(np.searchsorted(vit_cum, 0.90) + 1)
    W_vit   = pca_vit.components_[:vit_n]                             # (vit_n × D_vit)
    vit_scores_train = vit_train @ W_vit.T                             # (n_train × vit_n)

    # Brain PCA (samples=images, features=neurons)
    pca_brain = PCA(random_state=seed).fit(brain_train.T)
    brain_cum = np.cumsum(pca_brain.explained_variance_ratio_)
    brain_n   = int(np.searchsorted(brain_cum, 0.90) + 1)
    W_brain   = pca_brain.components_[:brain_n]                        # (brain_n × neurons)
    brain_scores_train = brain_train.T @ W_brain.T                     # (n_train × brain_n)
    return W_vit, W_brain, vit_scores_train, brain_scores_train, vit_n, brain_n

def discover_on_train(vit_scores_train, brain_scores_train, alpha=ALPHA_FDR):
    r_list, p_list, pairs = [], [], []
    vit_n = vit_scores_train.shape[1]
    brain_n = brain_scores_train.shape[1]
    for i in range(vit_n):
        for j in range(brain_n):
            r, p = pearsonr(vit_scores_train[:, i], brain_scores_train[:, j])
            r_list.append(r); p_list.append(p); pairs.append((i+1, j+1))  # 1-indexed for reporting
    r_arr = np.array(r_list); p_arr = np.array(p_list)
    reject, q_vals, _, _ = multipletests(p_arr, alpha=alpha, method='fdr_bh')
    sig_idx = np.where(reject)[0]
    return pairs, r_arr, p_arr, q_vals, sig_idx

def validate_on_test(W_vit, W_brain, vit_test, brain_test, sig_pairs):
    vit_scores_test   = vit_test @ W_vit.T
    brain_scores_test = brain_test.T @ W_brain.T
    out = []
    for (i,j) in sig_pairs:  # 1-indexed
        r, p = pearsonr(vit_scores_test[:, i-1], brain_scores_test[:, j-1])
        out.append((i, j, r, p))
    return out  # list of tuples

# -----------------------------
# 2-FOLD OUTER CV
# -----------------------------
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
fold_indices = list(kf.split(np.arange(N_IMAGES)))  # [(train_idx, test_idx), (..)]

all_rows = []

for fold_id, (train_idx, test_idx) in enumerate(fold_indices, 1):
    print(f"\n==== Fold {fold_id} (train {len(train_idx)}, test {len(test_idx)}) ====")
    vit_train, vit_test = vit_clr[train_idx], vit_clr[test_idx]
    brain_train, brain_test = X_mean[:, train_idx], X_mean[:, test_idx]

    # Fit PCA on TRAIN only
    W_vit, W_brain, vit_scores_train, brain_scores_train, vit_n, brain_n = fit_pca_train_only(vit_train, brain_train)
    print(f"  ViT train PCs (90%): {vit_n} | {AREA_NAME} train PCs (90%): {brain_n}")

    # Discovery on TRAIN with BH-FDR across ALL pairs
    pairs, r_tr, p_tr, q_tr, sig_idx = discover_on_train(vit_scores_train, brain_scores_train, alpha=ALPHA_FDR)
    print(f"  ✅ Train discovery significant pairs: {len(sig_idx)} (BH-FDR α={ALPHA_FDR})")

    sig_pairs = [pairs[k] for k in sig_idx]

    # Validate on the OPPOSITE fold’s TEST using current TRAIN bases (no refit)
    val_results = validate_on_test(W_vit, W_brain, vit_test, brain_test, sig_pairs)

    # Record rows for this direction (fold_id: train→its test)
    for k, (i, j) in enumerate(sig_pairs):
        all_rows.append({
            "direction": f"Fold{fold_id}_train→test",
            "fold": fold_id,
            "vit_pc_train": i,
            "brain_pc_train": j,
            "r_train": r_tr[sig_idx[k]],
            "p_train": p_tr[sig_idx[k]],
            "q_train": q_tr[sig_idx[k]],
            "r_test": val_results[k][2],
            "p_test": val_results[k][3],
            "n_train": len(train_idx),
            "n_test": len(test_idx),
        })

# -----------------------------
# AGGREGATION & SYMMETRIC REPLICATION
# -----------------------------
if not all_rows:
    print("\nNo train-significant pairs; nothing to report.")
else:
    df = pd.DataFrame(all_rows)
    df.to_csv(f"vit_{AREA_NAME}_2fold_raw.csv", index=False)
    print(f"\n💾 Saved per-direction raw results -> vit_{AREA_NAME}_2fold_raw.csv")

    # Split by direction
    d1 = df[df["direction"] == "Fold1_train→test"].copy()
    d2 = df[df["direction"] == "Fold2_train→test"].copy()

    # For symmetric replication, we need pairs that were discovered in Fold1 train and also discovered in Fold2 train,
    # and each validates on its own test. Because PCA bases differ per fold, we treat indices as fold-local;
    # we therefore define replication as: "exists any pair in Fold1 and any pair in Fold2 that both validate".
    # (If you want to *match* PCs across folds, add cosine-similarity matching on TRAIN loadings — leak-free.)

    # Mark validation success (test p < 0.05) per direction
    d1["validated"] = d1["p_test"] < 0.05
    d2["validated"] = d2["p_test"] < 0.05

    # Simple fold summaries
    fold1_sum = d1.agg(
        n_discovered=("p_train", "size"),
        n_validated=("validated", "sum"),
        mean_r_test=("r_test", "mean"),
        std_r_test=("r_test", "std")
    )
    fold2_sum = d2.agg(
        n_discovered=("p_train", "size"),
        n_validated=("validated", "sum"),
        mean_r_test=("r_test", "mean"),
        std_r_test=("r_test", "std")
    )
    summary = pd.DataFrame([fold1_sum, fold2_sum], index=["Fold1", "Fold2"])
    summary.to_csv(f"vit_{AREA_NAME}_2fold_fold_summary.csv")
    print("\n===== Per-fold summary =====")
    print(summary)

    # Extract lists of validated pairs per direction (fold-local indices)
    d1_val = d1[d1["validated"]][["vit_pc_train", "brain_pc_train", "r_test", "p_test"]].copy()
    d2_val = d2[d2["validated"]][["vit_pc_train", "brain_pc_train", "r_test", "p_test"]].copy()

    # Save validated lists
    d1_val.to_csv(f"vit_{AREA_NAME}_2fold_validated_fold1.csv", index=False)
    d2_val.to_csv(f"vit_{AREA_NAME}_2fold_validated_fold2.csv", index=False)

    # “Replicates both ways” here means: each fold produced at least one validated pair.
    # (Exact PC index matching across folds is not meaningful without alignment;
    # if you want alignment, add cosine-matching on TRAIN loadings and intersect.)
    replicated_both = (len(d1_val) > 0) and (len(d2_val) > 0)
    rep_df = pd.DataFrame({
        "replicated_both_ways": [bool(replicated_both)],
        "fold1_n_validated": [int(len(d1_val))],
        "fold2_n_validated": [int(len(d2_val))]
    })
    rep_df.to_csv(f"vit_{AREA_NAME}_2fold_replicated_flag.csv", index=False)
    print("\n===== Replication status =====")
    print(rep_df)
    print(f"\n💾 Saved summaries to files with prefix: vit_{AREA_NAME}_2fold_*")


🔹 Loading ViT embeddings...
🔹 Loading neural responses (VISam)...
VISam: 2040 neurons × 118 images

==== Fold 1 (train 94, test 24) ====
  ViT train PCs (90%): 38 | VISam train PCs (90%): 65
  ✅ Train discovery significant pairs: 0 (BH-FDR α=0.05)

==== Fold 2 (train 94, test 24) ====
  ViT train PCs (90%): 38 | VISam train PCs (90%): 65
  ✅ Train discovery significant pairs: 0 (BH-FDR α=0.05)

==== Fold 3 (train 94, test 24) ====
  ViT train PCs (90%): 39 | VISam train PCs (90%): 66
  ✅ Train discovery significant pairs: 0 (BH-FDR α=0.05)

==== Fold 4 (train 95, test 23) ====
  ViT train PCs (90%): 38 | VISam train PCs (90%): 67
  ✅ Train discovery significant pairs: 0 (BH-FDR α=0.05)

==== Fold 5 (train 95, test 23) ====
  ViT train PCs (90%): 39 | VISam train PCs (90%): 67
  ✅ Train discovery significant pairs: 1 (BH-FDR α=0.05)

💾 Saved per-direction raw results -> vit_VISam_2fold_raw.csv


IndexError: positional indexers are out-of-bounds